In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString, Point
import yaml 
import numpy as np
import folium
from pyproj import Proj, transform
from h3 import h3
import psycopg2
import matplotlib.pyplot as plt
%matplotlib inline

In [45]:
def visualizar_tramo_folium(tramos,tramo_id):

    tramo1 = tramos.loc[tramos.tramo_id==tramo_id,:].iloc[0]

    #si no hay origen siguiente, tomo el primero del dia 
    if tramo_id == tramos.tramo_id.iloc[-1]:
        origen_siguiente = tramos.tramo_id.iloc[0]
    else:
        origen_siguiente = tramo_id+1
        
    tramo2 = tramos.loc[tramos.tramo_id==origen_siguiente,:].iloc[0]
    
    
    if tramo1.h3_d is None:
        pass
    else:
        tramo1.lat_d = h3.h3_to_geo(tramo1.h3_d)[0]
        tramo1.lon_d = h3.h3_to_geo(tramo1.h3_d)[1]
        
    linea_trx = tramo1.linea
    
    if linea_trx in equivalencias.keys():
        linea_carto = equivalencias[linea_trx]
    else:
        linea_carto = linea_trx
    
    paradas_destino_posibles = paradas.loc[paradas.LINEA == linea_carto,['LATITUDE','LONGITUDE']]
    if tramo1.h3_d is None:
        miny = min(tramo1.lat_o, tramo2.lat_o) - 0.06
        maxy = max(tramo1.lat_o, tramo2.lat_o) + 0.06
        minx = min(tramo1.lon_o, tramo2.lon_o) - 0.06
        maxx = max(tramo1.lon_o, tramo2.lon_o) + 0.06
    else:
        miny = min(tramo1.lat_o, tramo2.lat_o) - 0.06
        maxy = max(tramo1.lat_o, tramo2.lat_o) + 0.06
        minx = min(tramo1.lon_o, tramo2.lon_o) - 0.06
        maxx = max(tramo1.lon_o, tramo2.lon_o) + 0.06


    x_centro,y_centro = minx + ((maxx - minx)/2),miny + ((maxy - miny)/2)

    m = folium.Map(
        location=[y_centro,x_centro],
        zoom_start=12,
        tiles='cartodbpositron'
    )
    
    for i in paradas_destino_posibles.index:
        folium.CircleMarker(
            radius=1,
            location=[paradas_destino_posibles.LATITUDE.loc[i],paradas_destino_posibles.LONGITUDE.loc[i]],
            color='black',
            fill=True,
            fill_color='black'
        ).add_to(m)
        
        
    folium.Circle(
        radius=100,
        location=[tramo1.lat_o,tramo1.lon_o],
        popup='Origen '+str(tramo1.fecha).split(' ')[1],
        color='crimson',
        fill=False,
    ).add_to(m)
    
    if tramo1.h3_d is None:
        pass
    else:

        folium.Marker(
            location=[tramo1.lat_d,tramo1.lon_d],
            popup='Destino',
            icon=folium.Icon(icon='glyphicon-remove-circle',color='red')
        ).add_to(m)
    
    
    folium.Circle(
        radius=100,
        location=[tramo2.lat_o,tramo2.lon_o],
        popup='Origen 2 '+str(tramo2.fecha).split(' ')[1],
        color='grey',
        fill=False,
    ).add_to(m)

                  
    return m

In [2]:
DB_USERNAME = 'sube_user'
DB_PASSWORD = 'sube_pass'
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'sube'
DB_SCHEMA = 'public'

In [3]:
resolucion = 11

In [4]:
with open('../recorridos_equivalencias.yaml') as file:
    equivalencias = yaml.load(file, Loader=yaml.FullLoader)

In [5]:
# Conectar a la db
conn = psycopg2.connect(user = DB_USERNAME,
                                      password = DB_PASSWORD,
                                      host = DB_HOST,
                                      port = DB_PORT,
                                      database = DB_NAME)

In [6]:
# traer trx de tarjetas que hayan pasado por la B
q = """
select distinct tarjeta
FROM tramos_linea_b
where h3_d is not null
"""
tarjetas_con_destino = pd.read_sql(q, conn)

In [27]:
#1263039 combina tren bus y subte
tarjeta = tarjetas_con_destino.sample(1).iloc[0,0]

In [28]:
q = """
select p."LINEA", p."LATITUDE",p."LONGITUDE",p."h3_res_%i"
from paradas p
"""%resolucion
paradas = pd.read_sql(q, conn)

In [32]:
q = """
select *
FROM tramos_linea_b
where tarjeta = %i
"""%tarjeta
tramos = pd.read_sql(q, conn)
tramos.sort_values('tramo_id')

,tarjeta,tramo_id,linea,fecha,lat_o,lon_o,delta,h3_o,lat_d,lon_d,h3_d
2,8884,0,LINEA B,2018-10-10 16:00:48,-34.603165,-58.420962,1.684000e+12,8bc2e311c075fff,None,None,8bc2e311015cfff
4,8884,1,SOFSE - Roca,2018-10-10 16:28:52,-34.628307,-58.380602,4.985000e+12,8bc2e3110156fff,None,None,8bc2e06cd85cfff
3,8884,2,LINEA 79,2018-10-10 17:51:57,-34.917330,-58.384720,8.130000e+11,8bc2e06cdb21fff,None,None,8bc2e06eb915fff
5,8884,3,LINEA 404,2018-10-10 18:05:33,-34.940180,-58.378500,5.278000e+12,8bc2e06eb932fff,None,None,None
1,8884,4,LINEA 51,2018-10-10 19:33:33,-34.835480,-58.387110,1.778000e+12,8bc2e3d6d191fff,None,None,8bc2e06eb141fff
7,8884,5,LINEA 51,2018-10-10 20:03:11,-34.931210,-58.379580,7.422000e+12,8bc2e06eb14afff,None,None,8bc2e3994711fff
6,8884,6,LINEA 79,2018-10-10 22:06:53,-34.882290,-58.390010,4.443000e+12,8bc2e3994715fff,None,None,8bc2e3110173fff
0,8884,7,LINEA 151,2018-10-10 23:20:56,-34.628690,-58.378720,NaN,8bc2e311011bfff,None,None,8bc2e311c15bfff


In [42]:
tramos.iloc[5]['h3_d'] is None

True

In [46]:
visualizar_tramo_folium(tramos,3)